In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import random

In [7]:
train_filepath = 'data/train.csv'
traindata = pd.read_table(train_filepath)

In [8]:
testdata = traindata.head()

In [9]:
testdata

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)
0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,33.0,33.0,NaN,1,0,0,1,NaN,NaN
1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,15.0,15.0,NaN,1,0,0,1,NaN,NaN
2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,78.0,NaN,78.0,0,2,0,1,Define Variable,1
3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2005-09-09 12:27:24.0,104.0,NaN,104.0,0,4,9,1,"Using small numbers~~Write expression, positiv...",1~~1~~1
4,9942,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R4C1,2005-09-09 12:27:24.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,2005-09-09 12:27:30.0,6.0,6.0,NaN,1,0,0,1,Entering a given,1


In [1]:
#get the pyspark part
import pyspark as ps

In [2]:
conf = ps.SparkConf()

In [4]:
sc = ps.SparkContext(conf = conf)

In [12]:
#Do a test for pyspark
filedata = sc.textFile("data/testspark.csv")

In [16]:
filedata.collect()

[',Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,Step Duration (sec),Correct Step Duration (sec),Error Step Duration (sec),Correct First Attempt,Incorrects,Hints,Corrects,KC(Default),Opportunity(Default)',
 '0,9938,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C1,2005-09-09 12:23:34.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,2005-09-09 12:24:07.0,33.0,33.0,,1,0,0,1,,',
 '1,9939,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R1C2,2005-09-09 12:24:07.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,2005-09-09 12:24:22.0,15.0,15.0,,1,0,0,1,,',
 '2,9940,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C1,2005-09-09 12:24:22.0,2005-09-09 12:25:16.0,2005-09-09 12:25:40.0,2005-09-09 12:25:40.0,78.0,,78.0,0,2,0,1,Define Variable,1',
 '3,9941,52vEY7f17k,"Unit CTA1_13, Section CTA1_13-1",PROP04,1,R3C2,2005-09-09 12:25:40.0,2005-09-09 12:25:51.0,2005-09-09 12:27:24.0,2

In [19]:
po = testdata["KC(Default)"]

In [20]:
po

0                                                  NaN
1                                                  NaN
2                                      Define Variable
3    Using small numbers~~Write expression, positiv...
4                                     Entering a given
Name: KC(Default), dtype: object

In [21]:
use = sc.parallelize(po)

In [75]:
#The initial part of the get function
import re
def get(x):
    if pd.isnull(x):
        return ["Nothing"]
    if "SkillRule" in x:
        filds = re.split("~~|;|:", x)
    else:
        filds = re.split(" |~~|;|:",x)
    newlist = list()
    for d in filds:
        deal = d
        deal = deal.strip()
        if "=" in deal:
            deal = deal.strip("{}]")
            newlist.append(deal)
        else:
            deal = deal.strip("{}[]")
            newlist.extend(deal.split())
    return newlist

def getword(x):
    filds = x.split(" ")
    return filds

kd = use.map(get)
print(kd.collect())

[['Nothing'], ['Nothing'], ['Define', 'Variable'], ['Using', 'small', 'numbers', 'Write', 'expression,', 'positive', 'slope', 'Using', 'difficult', 'numbers'], ['Entering', 'a', 'given']]


In [49]:
def fl(x):
    return x

kdd = kd.flatMap(fl)
kdd.countByValue()

defaultdict(int,
            {'Nothing': 2,
             'Define': 1,
             'Variable': 1,
             'Using': 2,
             'small': 1,
             'numbers': 2,
             'Write': 1,
             'expression,': 1,
             'positive': 1,
             'slope': 1,
             'difficult': 1,
             'Entering': 1,
             'a': 1,
             'given': 1})

In [47]:
kddd = kdd.map(getword)
kddd = kddd.flatMap(fl)
kddd.countByValue()

defaultdict(int,
            {'Nothing': 2,
             'Define': 1,
             'Variable': 1,
             'Using': 2,
             'small': 1,
             'numbers': 2,
             'Write': 1,
             'expression,': 1,
             'positive': 1,
             'slope': 1,
             'difficult': 1,
             'Entering': 1,
             'a': 1,
             'given': 1})

In [85]:
def listjoin1(x, y):
    op = list()
    op.extend(x)
    op.extend(y)
    return op

ttest2 = traindata.head(100)["KC(Default)"]
tpyuse = sc.parallelize(ttest2)
tmapre = tpyuse.map(get)
tredre = tmapre.reduce(listjoin1)
tredre = sc.parallelize(tredre)
tretul = tredre.countByValue()

In [86]:
tretul

defaultdict(int,
            {'Nothing': 33,
             'Define': 8,
             'Variable': 8,
             'Using': 42,
             'small': 21,
             'numbers': 42,
             'Write': 8,
             'expression,': 8,
             'positive': 13,
             'slope': 8,
             'difficult': 21,
             'Entering': 31,
             'a': 31,
             'given': 31,
             'Find': 13,
             'X,': 8,
             'Simple': 13,
             'Y,': 5,
             'SkillRule:': 12,
             'Remove': 11,
             'coefficient': 11,
             'ax/b=c, reciprocal': 11,
             'ax/b=c': 11,
             'x/a=b': 11,
             'ax=b': 17,
             'ax+b=c, divide': 6,
             '[const expr]*[var fact] + [const expr] = [const expr], divide': 6,
             '[var expr]*[const expr] = [const expr], divide': 6,
             'a/b*x=c': 6,
             'a/b*x=c, reciprocal': 6,
             '(+/-x +/-a)/b=c, mult': 7,
             

In [94]:
import re
def tget(x):
    if pd.isnull(x):
        return ["Nothing"]
    if "SkillRule" in x:
        filds = re.split("~~|;|:|{", x)
    else:
        filds = re.split(" |~~|;|:",x)
    newlist = list()
    for d in filds:
        deal = d
        deal = deal.strip()
        if "=" in deal:
            if deal[-2:] != "}]" and deal[-1] == "]" and " " not in deal:
                deal = deal.strip("{}")
            else:
                deal = deal.strip("{}]")
            newlist.append(deal)
        else:
            deal = deal.strip("{}[]")
            newlist.extend(deal.split())
    return newlist



def listjoin1(x, y):
    op = list()
    op.extend(x)
    op.extend(y)
    return op


ttest2 = traindata.head(100)["KC(Default)"]
tpyuse = sc.parallelize(ttest2)
tmapre = tpyuse.map(tget)
tredre = tmapre.reduce(listjoin1)
tredre = sc.parallelize(tredre)
tretul = tredre.countByValue()

In [95]:
tretul

defaultdict(int,
            {'Nothing': 33,
             'Define': 8,
             'Variable': 8,
             'Using': 42,
             'small': 21,
             'numbers': 42,
             'Write': 8,
             'expression,': 8,
             'positive': 13,
             'slope': 8,
             'difficult': 21,
             'Entering': 31,
             'a': 31,
             'given': 31,
             'Find': 13,
             'X,': 8,
             'Simple': 13,
             'Y,': 5,
             'SkillRule': 12,
             'Remove': 11,
             'coefficient': 11,
             'ax/b=c, reciprocal': 11,
             'ax/b=c': 11,
             'x/a=b': 11,
             'ax=b': 17,
             'ax+b=c, divide': 6,
             '[const expr]*[var fact] + [const expr] = [const expr], divide': 6,
             '[var expr]*[const expr] = [const expr], divide': 6,
             'a/b*x=c': 6,
             'a/b*x=c, reciprocal': 6,
             '(+/-x +/-a)/b=c, mult': 7,
             '

In [96]:
test2 = traindata["KC(Default)"]
pyuse = sc.parallelize(test2)
omapre = pyuse.map(tget)
oredre = omapre.reduce(listjoin1)
oredre = sc.parallelize(oredre)
rretul = oredre.countByValue()
rretul

defaultdict(int,
            {'Nothing': 59255,
             'Define': 7147,
             'Variable': 13057,
             'Using': 43480,
             'small': 14110,
             'numbers': 43480,
             'Write': 7993,
             'expression,': 7993,
             'positive': 28591,
             'slope': 19034,
             'difficult': 7576,
             'Entering': 27568,
             'a': 25036,
             'given': 22780,
             'Find': 13747,
             'X,': 7995,
             'Simple': 3606,
             'Y,': 5752,
             'SkillRule': 93919,
             'Remove': 57276,
             'coefficient': 36954,
             'ax/b=c, reciprocal': 32382,
             'ax/b=c': 32382,
             'x/a=b': 32382,
             'ax=b': 51027,
             'ax+b=c, divide': 18645,
             '[const expr]*[var fact] + [const expr] = [const expr], divide': 18645,
             '[var expr]*[const expr] = [const expr], divide': 18772,
             'a/b*x=c': 18645,
   

In [119]:
ui = dict(rretul)
ui

{'Nothing': 59255,
 'Define': 7147,
 'Variable': 13057,
 'Using': 43480,
 'small': 14110,
 'numbers': 43480,
 'Write': 7993,
 'expression,': 7993,
 'positive': 28591,
 'slope': 19034,
 'difficult': 7576,
 'Entering': 27568,
 'a': 25036,
 'given': 22780,
 'Find': 13747,
 'X,': 7995,
 'Simple': 3606,
 'Y,': 5752,
 'SkillRule': 93919,
 'Remove': 57276,
 'coefficient': 36954,
 'ax/b=c, reciprocal': 32382,
 'ax/b=c': 32382,
 'x/a=b': 32382,
 'ax=b': 51027,
 'ax+b=c, divide': 18645,
 '[const expr]*[var fact] + [const expr] = [const expr], divide': 18645,
 '[var expr]*[const expr] = [const expr], divide': 18772,
 'a/b*x=c': 18645,
 'a/b*x=c, reciprocal': 18645,
 '(+/-x +/-a)/b=c, mult': 24182,
 'a=x*(b+c)': 18645,
 'a=x*(b-c)': 18645,
 'a=x*(b*c+d)': 18645,
 'x/a+b=c, multiply': 18645,
 '[var expr]/[const expr] = [const expr], multiply': 24309,
 'Identifying': 10760,
 'units': 10760,
 'Eliminate': 9725,
 'Parens': 9725,
 'CLT': 28983,
 'nested': 10169,
 'nested,': 8316,
 'parens': 8316,
 'Dis

In [118]:
lp = pd.DataFrame(list(ui.items()),columns = ["KC Name", "Appear times"])
lp.to_csv("data/mapreduceresult.csv")